In [2]:
import instaloader
import time
from duckduckgo_search import DDGS
from IPython.display import clear_output
import httpx
import pandas as pd
import numpy as np

In [163]:
# Get the username by scraping using Duckduck Go search api

In [22]:
social_media = pd.read_csv('credibility.csv', index_col=None).drop(columns = 'Unnamed: 0')
social_media

,source,credibility_score,source_history
0,Instagram posts,1.74,Instagram List of most-followed Instagram acco...
1,Scott Walker,5.14,Scott Walker Scott Walker (politician) Scott W...
2,Viral image,1.60,Viral phenomenon Imgur The dress Midjourney Da...
3,Facebook posts,1.86,Facebook Facebook 3D Posts Meta Platforms Hist...
4,New York Citizens Audit,2.00,Sovereign citizen movement KPMG BNY Mellon Arc...
...,...,...,...
4695,Freedom Defense Advocates,0.00,United States Army Judge Advocate General's Co...
4696,Spike the Romney Attack Dog,8.00,Mitt Romney List of individual dogs January 6 ...
4697,Log Cabin Republicans,8.00,Log Cabin Republicans Don Korotsky Norte Log C...
4698,Sam Brownback,6.50,Sam Brownback Kansas experiment Sam Brownback ...


In [23]:
source_lst = np.array((social_media['source'].str.replace(' posts', '')))
source_lst[source_lst == 'X'] = 'Twitter'

In [335]:
# Improved Scrape Algorithm
instagram_urls = []
i = 0
failed_index = []
for name in source_lst:
    try:
        with DDGS() as ddgs:
            results = [r for r in ddgs.text(name + " instagram account", max_results=10)]    
            instagram_account = None
            for item in results:
                if 'instagram.com' in item['href'] and '@' in item['title']:
                    instagram_account = item['href']
                    break
            instagram_urls.append(instagram_account)
            if instagram_account == None:
                failed_index.append(i)
    except httpx.HTTPError as e:
        print(f"Fail to retrieve at ",i, 'with name', name)
        instagram_urls.append(None)
        failed_index.append(i)
    if i % 100 == 0:
            print(f"Iteration", i, "is done")
            clear_output(wait=True)
    i+=1
social_media['instagram_uri'] = np.array(instagram_urls)
# social_media.to_csv('credibility_with_insta_username.csv')

Fail to retrieve at  4613 with name America's Health Insurance Plans
Fail to retrieve at  4638 with name America's Natural Gas Alliance
Fail to retrieve at  4673 with name Conservatives for Patients' Rights


In [19]:
social_media = pd.read_csv('credibility_with_insta_username.csv').drop(columns = 'Unnamed: 0')
cleaned_social_media = social_media[social_media['instagram_uri'].str.contains('instagram', na=False)].reset_index().drop(columns = 'index')

In [20]:
from urllib.parse import urlparse
def get_username(instagram_url):
    parsed_url = urlparse(instagram_url)
    path_segments = parsed_url.path.strip('/').split('/')
    username = path_segments[-1]
    return username

In [21]:
cleaned_social_media['username'] = cleaned_social_media['instagram_uri'].apply(get_username)

In [22]:
cleaned_social_media

,source,credibility_score,source_history,instagram_uri,username
0,Instagram posts,1.74,Instagram List of most-followed Instagram acco...,https://www.instagram.com/instagram/,instagram
1,Scott Walker,5.14,Scott Walker Scott Walker (politician) Scott W...,https://www.instagram.com/scottwalker/,scottwalker
2,New York Citizens Audit,2.00,Sovereign citizen movement KPMG BNY Mellon Arc...,https://www.instagram.com/newyorkcitizensaudit...,newyorkcitizensauditnews
3,Steve Kirsch,0.80,Steve Kirsch Stan Kirsch Against All Odds (198...,https://www.instagram.com/steventkirsch/,steventkirsch
4,Shannon Watts,6.00,Shannon Watts Watts family murders Shannon Wat...,https://www.instagram.com/shannonrwatts/,shannonrwatts
...,...,...,...,...,...
4005,Freedom Defense Advocates,0.00,United States Army Judge Advocate General's Co...,https://www.instagram.com/freedomguards/,freedomguards
4006,Spike the Romney Attack Dog,8.00,Mitt Romney List of individual dogs January 6 ...,https://www.instagram.com/spikeakanickilrettil...,spikeakanickilrettiliano
4007,Log Cabin Republicans,8.00,Log Cabin Republicans Don Korotsky Norte Log C...,https://www.instagram.com/logcabincolorado/,logcabincolorado
4008,Sam Brownback,6.50,Sam Brownback Kansas experiment Sam Brownback ...,https://www.instagram.com/govbrownback/,govbrownback


In [23]:
def scrape_account(username, bot):
    try:
        caption_lst = []
        profile = instaloader.Profile.from_username(bot.context, username)
        print(f"Profile '{username}' exists.")
        time.sleep(5)
        return profile.username, profile.biography, profile.userid, profile.mediacount, profile.followers, profile.followees, profile.is_verified#, caption_lst
    
    except instaloader.ProfileNotExistsException:
        print(f"Profile '{username}' does not exist.")
        return None
bot = instaloader.Instaloader()
instaloader.Instaloader.login(bot, user = 'seanjiang10', passwd = '1037635008')

In [16]:
usernames = np.array(cleaned_social_media['instagram_username'])
i = 0
account_info_lst = []
for name in usernames:
    result = scrape_account(name, bot)
    print(f"Iteration", i, "is done")
    account_info_lst.append(result)
    if i % 10 == 0:
        clear_output(wait=True)
    i += 1
# reliability_df = pd.DataFrame(account_info_lst, columns = ['username', 'biography','userid','mediacount','followers','followees','is_verified'])
# reliability_df.to_csv('reliability.csv')

Profile 'libertyguardofficial' exists.
Iteration 4001 is done
Profile 'focusonthefamily' exists.
Iteration 4002 is done
Profile 'mike.gravel' exists.
Iteration 4003 is done
Profile 'psychicmediumje' exists.
Iteration 4004 is done
Profile 'freedomguards' exists.
Iteration 4005 is done
Profile 'spikeakanickilrettiliano' exists.
Iteration 4006 is done
Profile 'logcabincolorado' exists.
Iteration 4007 is done
Profile 'govbrownback' exists.
Iteration 4008 is done
Profile 'michelleobama' exists.
Iteration 4009 is done


In [31]:
insta_df = pd.read_csv('reliability.csv').drop(columns = 'Unnamed: 0')
insta_df = insta_df.drop_duplicates(subset='username', keep='last')
insta_df

,username,biography,userid,mediacount,followers,followees,is_verified
0,scottwalker,Christian American Husband Father 45th Governo...,586553277,8683,47989,175,True
1,newyorkcitizensauditnews,NY Citizens Audit is a Volunteer based group. ...,50346417000,1202,891,460,False
2,steventkirsch,Retired serial entrepreneur. Follow me on stev...,184544694,10,4469,20,False
3,shannonrwatts,"🎤 Speaker, author, organizer\n🙋‍♀️ “Summoner o...",24779854,431,129536,2369,True
4,chris_larson,NaN,5941863,1138,1743,349,False
...,...,...,...,...,...,...,...
3669,freedomguards,# 1 In Exposing Voices of the Unheard Patriots...,33533983021,388,69452,335,False
3670,spikeakanickilrettiliano,#punge feat @krin183dsacommando prod @sundayd...,1806764556,275,2290,1177,False
3671,logcabincolorado,Official Instagram account of the Log Cabin Re...,45809376743,868,1026,696,False
3672,govbrownback,Official campaign account for Kansas Governor ...,1467894876,63,515,76,False


In [35]:
unique_df = pd.DataFrame({"username" : np.unique(cleaned_social_media[['username']])})
merged_df = pd.merge(insta_df, unique_df, on='username', how='inner').reset_index().drop(columns = 'index')
merged_df

,username,biography,userid,mediacount,followers,followees,is_verified
0,scottwalker,Christian American Husband Father 45th Governo...,586553277,8683,47989,175,True
1,newyorkcitizensauditnews,NY Citizens Audit is a Volunteer based group. ...,50346417000,1202,891,460,False
2,steventkirsch,Retired serial entrepreneur. Follow me on stev...,184544694,10,4469,20,False
3,shannonrwatts,"🎤 Speaker, author, organizer\n🙋‍♀️ “Summoner o...",24779854,431,129536,2369,True
4,chris_larson,NaN,5941863,1138,1743,349,False
...,...,...,...,...,...,...,...
3669,freedomguards,# 1 In Exposing Voices of the Unheard Patriots...,33533983021,388,69452,335,False
3670,spikeakanickilrettiliano,#punge feat @krin183dsacommando prod @sundayd...,1806764556,275,2290,1177,False
3671,logcabincolorado,Official Instagram account of the Log Cabin Re...,45809376743,868,1026,696,False
3672,govbrownback,Official campaign account for Kansas Governor ...,1467894876,63,515,76,False


In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler

features = merged_df[['mediacount', 'followers', 'followees', 'is_verified']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
agglomerative = AgglomerativeClustering(n_clusters=None, distance_threshold=10, linkage='ward')
merged_df['reliability_score'] = agglomerative.fit_predict(scaled_features)
min_max_scaler = MinMaxScaler(feature_range=(1, 5))
merged_df['reliability_score'] = min_max_scaler.fit_transform(merged_df['reliability_score'].values.reshape(-1, 1))
merged_df[['username', 'reliability_score']]

,username,reliability_score
0,scottwalker,1.000000
1,newyorkcitizensauditnews,2.333333
2,steventkirsch,2.333333
3,shannonrwatts,3.444444
4,chris_larson,2.333333
...,...,...
3669,freedomguards,2.333333
3670,spikeakanickilrettiliano,3.888889
3671,logcabincolorado,2.333333
3672,govbrownback,2.333333
